In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from matplotlib import rc
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from datetime import datetime
import numpy as np
# from sklearn.datasets import load_breast_cancer
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split

plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='AppleGothic'
#https://teddylee777.github.io/visualization/matplotlib-%EC%8B%9C%EA%B0%81%ED%99%94-%ED%95%9C%EA%B8%80%ED%8F%B0%ED%8A%B8%EC%A0%81%EC%9A%A9
%matplotlib inline 

In [5]:
temp_14=pd.read_csv('2014년도 기온.csv',encoding='cp949').iloc[:10000,:]
temp_14

,지점,지점명,일시,기온(°C)
0,90,속초,2014-01-01 00:00,6.5
1,90,속초,2014-01-01 01:00,5.6
2,90,속초,2014-01-01 02:00,5.2
3,90,속초,2014-01-01 03:00,5.0
4,90,속초,2014-01-01 04:00,5.2
...,...,...,...,...
9995,95,철원,2014-02-21 11:00,1.2
9996,95,철원,2014-02-21 12:00,3.0
9997,95,철원,2014-02-21 13:00,3.3
9998,95,철원,2014-02-21 14:00,4.2


In [6]:
temp_14.isnull().sum()

지점        0
지점명       0
일시        0
기온(°C)    0
dtype: int64

In [7]:
temp_14["일시"] = temp_14["일시"].apply(lambda x : datetime.strptime(x,'%Y-%m-%d %H:%M'))
temp_14["년"] = temp_14["일시"].apply(lambda x : x.year)
temp_14["월"] = temp_14["일시"].apply(lambda x : x.month)
temp_14["일"] = temp_14["일시"].apply(lambda x : x.day)
temp_14["시"] = temp_14["일시"].apply(lambda x : x.hour)
temp_14["분"] = temp_14["일시"].apply(lambda x : x.minute)
temp_14["일자"]=temp_14["일시"].dt.date
temp_14

,지점,지점명,일시,기온(°C),년,월,일,시,분,일자
0,90,속초,2014-01-01 00:00:00,6.5,2014,1,1,0,0,2014-01-01
1,90,속초,2014-01-01 01:00:00,5.6,2014,1,1,1,0,2014-01-01
2,90,속초,2014-01-01 02:00:00,5.2,2014,1,1,2,0,2014-01-01
3,90,속초,2014-01-01 03:00:00,5.0,2014,1,1,3,0,2014-01-01
4,90,속초,2014-01-01 04:00:00,5.2,2014,1,1,4,0,2014-01-01
...,...,...,...,...,...,...,...,...,...,...
9995,95,철원,2014-02-21 11:00:00,1.2,2014,2,21,11,0,2014-02-21
9996,95,철원,2014-02-21 12:00:00,3.0,2014,2,21,12,0,2014-02-21
9997,95,철원,2014-02-21 13:00:00,3.3,2014,2,21,13,0,2014-02-21
9998,95,철원,2014-02-21 14:00:00,4.2,2014,2,21,14,0,2014-02-21


## 일부 '시(시간)' 데이터가 누락된 것으로 보임, 근데 아예 행 자체가 없어서 NAN으로 측정되지는 않음

In [8]:
def 지역별일평균기온결측(시도,df): 

    # 딕셔너리 형태로 시도별 지점 정보저장
    loc_dic = {"강원":["강릉","대관령","동해","북강릉","속초","영월","원주","인제","정선군","철원","춘천","태백","홍천"],
              "경기":["동두천","수원","양평","이천","파주"],
              "경남":["거제", "거창", "김해시", "남해", "밀양", "북창원", "산청", "양산시", "의령군", "진주", "창원", "통영", "함양군", "합천"],
              "경북":["경주시", "구미", "문경", "봉화", "상주", "안동", "영덕", "영주", "영천", "울릉도", "울진", "의성", "청송군", "포항"],
              "광주":["광주"], "대구":["대구"],"대전":["대전"], "부산":["부산"],"서울":["관악산","서울"],"울산":["울산"],
              "인천": ["강화","백령도","인천"],
              "전남":["강진군", "고흥" ,"광양시", "목포", "보성군", "순천", "여수", "영광군", "완도", "장흥" ,"주암", "진도(첨찰산)","해남" ,"흑산도"],
              "전북":["고창", "고창군", "군산", "남원", "부안", "순창군", "임실", "장수", "전주", "정읍"],
              "제주":["고산", "서귀포", "성산","제주"],
              "충남":["금산", "보령", "부여", "서산", "천안"], "충북":["보은", "제천", "청주", "추풍령", "충주"]}
    
    
    count=0 ## 결측값의 갯수세기
    for loc in loc_dic[시도]:
        for i in range(1,13):
            dam=df[(df["지점명"]==loc)&(df["월"]==i)]
            ## 해당 '월'의 '일'수를 카운트하기 위해서
            days=dam["일"].to_list()   
            day=list(np.unique(days))
            
            for j in day:
                for k in range(0,24):
                    dop=dam[dam["일"]==j]
                    times=dop["시"].to_list()
                    time=list(np.unique(times))
                    if not k in time:
                        s="{0}의 없는 날은 {1}월 {2}일 {3}시 입니다.".format(loc,i,j,k)
                        count+=1
                        print(s)
                        
    ss="결측값의 갯수: {0}개".format(count)
    print(ss)           

In [9]:
지역별일평균기온결측("강원",temp_14)   ## 결측값 존재  강릉,동해,인제, 정선군, 철원, 춘천, 태백     ##38개
#지역별일평균기온결측("경기",temp_14)    ## 결측값 존재  동두천, 양평, 파주                     ##10개
#지역별일평균기온결측("경남",temp_14)    ## 결측값 존재 김해시, 남해, 북창원, 의령군, 함양군        ##52개
#지역별일평균기온결측("경북",temp_14)    ## 결측값 존재 문경, 봉화, 상주, 영덕, 울진              ##26개
#지역별일평균기온결측("광주",temp_14)    ## 결측값 존재안함
#지역별일평균기온결측("대구",temp_14)    ## 결측값 존재안함
#지역별일평균기온결측("대전",temp_14)    ## 결측값 존재안함
#지역별일평균기온결측("부산",temp_14)    ## 결측값 존재안함
#지역별일평균기온결측("서울",temp_14)    ## 결측값 존재안함
#지역별일평균기온결측("울산",temp_14)    ## 결측값 존재안함
#지역별일평균기온결측("인천",temp_14)    ## 결측값 존재 강화, 백령도                           ##3개
#지역별일평균기온결측("전남",temp_14)    ## 결측값 존재 강진군, 광양시, 순천, 영광군, 해남         ##18개
#지역별일평균기온결측("전북",temp_14)    ## 결측값 존재 순창군, 임실, 정읍                      ##16개
#지역별일평균기온결측("제주",temp_14)    ## 결측값 존재 고산, 서귀포                           ##2개
#지역별일평균기온결측("충남",temp_14)    ## 결측값 존재 금산, 천안                            ##9개
#지역별일평균기온결측("충북",temp_14)    ## 결측값 존재 보은                                  ##3개

#local=["강원","경기","경남","경북","광주","대구","대전","부산","서울","울산","인천","전남","전북","제주","충남","충북"]
#for i in local:
    #지역별일평균기온결측(i,temp_14)

철원의 없는 날은 2월 21일 16시 입니다.
철원의 없는 날은 2월 21일 17시 입니다.
철원의 없는 날은 2월 21일 18시 입니다.
철원의 없는 날은 2월 21일 19시 입니다.
철원의 없는 날은 2월 21일 20시 입니다.
철원의 없는 날은 2월 21일 21시 입니다.
철원의 없는 날은 2월 21일 22시 입니다.
철원의 없는 날은 2월 21일 23시 입니다.
결측값의 갯수: 8개


DatetimeIndex(['2014-01-01 00:00:00', '2014-01-01 01:00:00',
               '2014-01-01 02:00:00', '2014-01-01 03:00:00',
               '2014-01-01 04:00:00', '2014-01-01 05:00:00',
               '2014-01-01 06:00:00', '2014-01-01 07:00:00',
               '2014-01-01 08:00:00', '2014-01-01 09:00:00',
               ...
               '2014-12-30 15:00:00', '2014-12-30 16:00:00',
               '2014-12-30 17:00:00', '2014-12-30 18:00:00',
               '2014-12-30 19:00:00', '2014-12-30 20:00:00',
               '2014-12-30 21:00:00', '2014-12-30 22:00:00',
               '2014-12-30 23:00:00', '2014-12-31 00:00:00'],
              dtype='datetime64[ns]', length=8737, freq='H')

In [17]:
# 1월 1일 0시부터 ~ 12월 31일 0시까지
# 2014년의 모든 시간이 담긴 데이터프레임
# 해당 데이터프레임을 통해 결측이 있는지 확인

date_df = pd.DataFrame(data=pd.date_range("2014-01-01","2014-12-31",freq = "H"),
                      columns = ["일시"])

In [18]:
# 특정 지점의 데이터만 추출

속초 = temp_14[temp_14["지점명"] == "속초"]

In [21]:
# 데이터프레임 2개 연결

date_df_속초 = pd.merge(date_df,속초[["지점명","일시","기온(°C)"]],
                      on="일시",how="left")


In [22]:
# 결측치 확인 -> 속초는 결측이 없는 지역
date_df_속초.isnull().sum()

일시        0
지점명       0
기온(°C)    0
dtype: int64

### 결측값의 비율이 5~10% 이내면 단일대체를 하고 그 이상은 더 정교한 대체/// 30% 이상이면 대체하는 것이 의미가 없을 수도 있다

In [ ]:
aa=[38,10,52,26,3,18,16,2,9,3]   ## 지역 별 결측값 ㄱ
sum(aa)/len(temp_14)*100